In [36]:
import json
import os
GT_JSON='/data/murayama/k8s/ocr_dxs1/donut/dataset/kie/invoice_nttd/valid_data.json'
PRED_DIR='/data/murayama/k8s/ocr_dxs1/donut/result/test_invoice'

pred_json_list = os.listdir(PRED_DIR)
pred_json_list = [os.path.join(PRED_DIR, jj) for jj in pred_json_list if '.json' in jj]
pred_data_list = {}
for jj in pred_json_list:
    fn = os.path.basename(jj)
    fn = os.path.splitext(fn)[0]
    with open(jj) as fp:
        data = json.load(fp)
    pred_data_list[fn] = data


with open(GT_JSON) as fp:
    gt_data = json.load(fp)


In [37]:
eval_data_list = {}
keys = [
"Issue_Time",
"Non_Reduced_Tax",
"Reduced_Tax",
"Page_Number",
"Office_Name",
"Biller_Fax",
"Currency",
"Issue_Month",
"Subtotal",
"Account_Name",
"Account_Number",
"Account_Type",
"Bankbranch_Name",
"Bank_Name",
"Due_Date",
"Consumption_Tax",
"Total_Amount",
"Billing_Company",
"Biller_Tel",
"Biller_Address",
"Biller_Company",
"Title",
"Invoice_Number",
"Issue_Date",
"Reduced_Subtotal",
"Non_Reduced_Subtotal",
"Register_No",]

for ind in gt_data:
    eval_data = {kk:{'gt':'', 'pred':''} for kk in keys}
    items = ind['gt_parse']['invoice']
    fn = ind['image']
    fn = os.path.splitext(os.path.basename(fn))[0]
    preds = pred_data_list[fn]
    print(fn)
    if isinstance(preds, dict):
        preds = [preds,]
    for ii in items:
        kk = list(ii.keys())[0]
        eval_data[kk]['gt'] = ii[kk].replace(' ','').replace('(','')

    for ii in preds:
        kk = list(ii.keys())[0]
        if not kk in keys:
            continue
        if isinstance(ii[kk]['content'], str):
            val = ii[kk]['content'].replace('登録番号','').replace(' ','')
        else:
            val = ''
        eval_data[kk]['pred'] = val

    for kk in eval_data:
        isok = eval_data[kk]['gt'] == eval_data[kk]['pred']
        eval_data[kk]['eval'] = isok

    eval_data_list[fn] = eval_data


202307_nttd_請求書_テンプレート-003_郵送.pdf-1
202307_nttd_川口土建指定請求書用紙（数式有り）.pdf-1
202307_nttd_三神工業㈱　指定請求書.pdf-1
202307_nttd_パターン３_請求先_請求先会社.pdf-1
202307_nttd_パターン２_請求先_取引太郎.pdf-1
202307_nttd_パターン１_請求先_株式会社テスト.pdf-1
202307_nttd_seikyuusyo_kouji_invoice_202303.pdf-1
202307_nttd_seikyuuippanekuseru.pdf-1
202307_nttd_seikyusyoo.pdf-1
202307_nttd_seikyusyo_invoice_excel.pdf-4
202307_nttd_seikyusyo_invoice_excel.pdf-3
202307_nttd_seikyusyo_invoice_excel.pdf-2
202307_nttd_seikyusyo_invoice_excel.pdf-1
202307_nttd_seikyusyo.pdf-1
202307_nttd_seikyusho_10.pdf-4
202307_nttd_seikyusho_10.pdf-3
202307_nttd_seikyukensyu.pdf-1
202307_nttd_seikyu_zairyokeihi_20230328_P43のみ.pdf-43
202307_nttd_keiyo-koukan_invoice.pdf-1
202307_nttd_invoice_seikyusho.pdf-1
202307_nttd_invoice_01.pdf-1
202307_nttd_invoice20201204-tax10.pdf-1
202307_nttd_invoice02_excel.pdf-1
202307_nttd_excel_shitei-Invoice.pdf-1
202307_nttd_e377a6763f92eb351e17f1397ecf55fb.pdf-3
202307_nttd_e377a6763f92eb351e17f1397ecf55fb.pdf-2
202307_nttd_e

In [38]:
eval_data_list

{'202307_nttd_請求書_テンプレート-003_郵送.pdf-1': {'Issue_Time': {'gt': '',
   'pred': '',
   'eval': True},
  'Non_Reduced_Tax': {'gt': '', 'pred': '0', 'eval': False},
  'Reduced_Tax': {'gt': '80', 'pred': '10,000', 'eval': False},
  'Page_Number': {'gt': '', 'pred': '', 'eval': True},
  'Office_Name': {'gt': '', 'pred': '', 'eval': True},
  'Biller_Fax': {'gt': '', 'pred': '', 'eval': True},
  'Currency': {'gt': '¥', 'pred': '', 'eval': False},
  'Issue_Month': {'gt': '2021年12月分', 'pred': '', 'eval': False},
  'Subtotal': {'gt': '', 'pred': '600-0031', 'eval': False},
  'Account_Name': {'gt': 'テキカクショカ', 'pred': 'テキカクシュ(カ', 'eval': False},
  'Account_Number': {'gt': '1234567', 'pred': '1234567', 'eval': True},
  'Account_Type': {'gt': '普通', 'pred': '普通', 'eval': True},
  'Bankbranch_Name': {'gt': '東大阪支店', 'pred': '東大阪支店', 'eval': True},
  'Bank_Name': {'gt': 'みずほ銀行', 'pred': 'みずほ銀行', 'eval': True},
  'Due_Date': {'gt': '2024/1/15', 'pred': '2021年12月20日', 'eval': False},
  'Consumption_Tax': {

In [39]:
keywise = {kk:{'OK':0, 'NG':0} for kk in keys}
imgwise = {fn:{'OK':0, 'NG':0} for fn in pred_data_list}

for fn in eval_data_list:
    for kk in eval_data_list[fn]:
        is_ok = eval_data_list[fn][kk]['eval']
        if is_ok:
            keywise[kk]['OK'] += 1
            imgwise[fn]['OK'] += 1
        else:
            keywise[kk]['NG'] += 1
            imgwise[fn]['NG'] += 1

with open('invoice_eval.csv', 'w') as fp:
    for kk in keywise:
        acc = keywise[kk]['OK'] / (keywise[kk]['OK']+keywise[kk]['NG'])
        keywise[kk]['ACC'] = acc
        fp.write('{},{}\n'.format(kk, acc))

    for fn in imgwise:
        acc = imgwise[fn]['OK'] / (imgwise[fn]['OK']+imgwise[fn]['NG'])
        imgwise[fn]['ACC'] = acc
        fp.write('{},{}\n'.format(fn, acc))


In [40]:
with open('invoice_eval.json', 'w') as fp:
    json.dump(eval_data_list, fp, ensure_ascii=False, indent=4)